In [2]:
import os

In [3]:
%pwd

'e:\\Risk-Analysis\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'e:\\Risk-Analysis'

In [6]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelTrainerConfig:
    trainer_dir  : Path
    model_dir  : Path
    transformation_dir :Path
    preprocessor_dir :Path


In [7]:
from src.Risk_Analysis.constants import config_file
from src.Risk_Analysis.constants import params_file
from src.Risk_Analysis.utils.common import real_yaml,create_directories

In [8]:
class Configuration:
    def __init__(self,config=config_file,params=params_file):
        self.config=real_yaml(config)
        self.params=real_yaml(params)
        create_directories([self.config.Artifacts])

    def model_trainer_config(self)->ModelTrainerConfig:
        trainer=self.config.Model_Trainer
        transformation=self.config.Data_Transformation
        

        create_directories([trainer.trainer_dir])

        trainer_config=ModelTrainerConfig(
            trainer_dir=trainer.trainer_dir,
            model_dir=trainer.model_dir,
             transformation_dir=transformation.transformation_dir,
            preprocessor_dir=transformation.preprocessor_dir,

            
        )
        return trainer_config

In [9]:
import pandas as pd
import numpy as np
from src.Risk_Analysis.utils.common import save_oject,model_eval

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBRFClassifier
from catboost import CatBoostClassifier
from src.Risk_Analysis.utils.common import model_eval

In [22]:
class ModelTrainer:
    def __init__(self,config):
        self.config=config

    def get_model_trainer(self,x_train,x_test,y_train,y_test):

        models={'logistic_model':LogisticRegression(),
                'Naie_bayes_model':GaussianNB(),
                'K_Neighbors_model':KNeighborsClassifier(),
                'Decision_tree_model':DecisionTreeClassifier(),
                'Random_forest':RandomForestClassifier(),
                'GradientBoostingClassifier':GradientBoostingClassifier(),
                'xgboost':XGBRFClassifier(),
                'cat_boost':CatBoostClassifier()}
        
        recall_score_list=[]
        accuracy_score_list=[]
        model_list=[]
        for i in range(len(list(models))):
            model=list(models.values())[i]
            model.fit(x_train,y_train)

            predict=model.predict(x_test)

            score,recall=model_eval(y_test,predict)

            model_list.append(list(models.keys())[i])

            accuracy_score_list.append(score)
            recall_score_list.append(recall)
       
        return (recall_score_list,models,model_list)
    
    def select_best_model(self,x_train,x_test,y_train,y_test):
        report={}

        recall_score_list,models,model_list=self.get_model_trainer(x_train=x_train,x_test=x_test,y_train=y_train,y_test=y_test)

        for key,value in zip(model_list,recall_score_list):
            report[key]=value

        sorted_model=sorted(report.items(),key=lambda x:x[1],reverse=True)
        print(sorted_model)

        model=sorted_model[0][0]

        best_recall_score=sorted_model[0][1]

        best_model=models[model]

        save_oject(filepath=self.config.model_dir,obj=best_model)

        print(best_recall_score)
      

In [23]:
from src.Risk_Analysis.components.data_transformation import DataTransformation

In [24]:
try:
    trainer=Configuration()
    config=trainer.model_trainer_config()

    transformation=DataTransformation(config)
    x_train,x_test,y_train,y_test=transformation.data_transformation_initiated()

    model_trainer=ModelTrainer(config)

    model=model_trainer.select_best_model(x_train,x_test,y_train,y_test)

except Exception as e:
    raise e


[2023-09-24 17:39:50,460 - Risk Analysis - INFO - directory is created ]
[2023-09-24 17:39:50,471 - Risk Analysis - INFO - directory is created ]


c:\Users\HOME\anaconda3\envs\Risk\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Learning rate set to 0.168482
0:	learn: 0.5336031	total: 266ms	remaining: 4m 25s
1:	learn: 0.4284822	total: 536ms	remaining: 4m 27s
2:	learn: 0.3722351	total: 894ms	remaining: 4m 56s
3:	learn: 0.3352746	total: 1.15s	remaining: 4m 45s
4:	learn: 0.3130068	total: 1.35s	remaining: 4m 27s
5:	learn: 0.2952124	total: 1.56s	remaining: 4m 17s
6:	learn: 0.2848771	total: 1.76s	remaining: 4m 9s
7:	learn: 0.2698749	total: 1.99s	remaining: 4m 6s
8:	learn: 0.2614185	total: 2.22s	remaining: 4m 4s
9:	learn: 0.2558049	total: 2.42s	remaining: 3m 59s
10:	learn: 0.2513825	total: 2.66s	remaining: 3m 59s
11:	learn: 0.2475784	total: 2.85s	remaining: 3m 54s
12:	learn: 0.2443570	total: 3.07s	remaining: 3m 53s
13:	learn: 0.2382317	total: 3.33s	remaining: 3m 54s
14:	learn: 0.2362813	total: 3.54s	remaining: 3m 52s
15:	learn: 0.2344329	total: 3.78s	remaining: 3m 52s
16:	learn: 0.2316280	total: 3.98s	remaining: 3m 49s
17:	learn: 0.2297123	total: 4.21s	remaining: 3m 49s
18:	learn: 0.2258643	total: 4.41s	remaining: 3m